In [1]:
import os
import sys

import sagemaker
from sagemaker import get_execution_role

# Add the parent directory to the sys.path
sys.path.insert(0, os.path.abspath(".."))

# Define IAM role
role = get_execution_role()
role

# Establish S3 bucket connection
import boto3

s3 = boto3.client("s3")
bucket = "capstone-bucket-4-friends"

# Take a look at current dir
print(os.getcwd())

from file_utilities import s3_download

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
/home/sagemaker-user/capstone-2024-summer/src/jenna/phase_1


In [2]:
!pip install seaborn
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.preprocessing import OneHotEncoder

  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)


In [3]:
# reload_path = s3_download("CRSP/crsp_2018-2023_clean_3.parquet")
# reload_path

In [4]:
df = pd.read_parquet("/home/sagemaker-user/capstone-2024-summer/data/crsp_2018-2023_clean_3.parquet")

In [5]:
df

,date,permno_id,ncusip_id,cusip_id,ticker,share_code,share_code_type,share_code_detail,exchange_code,company_name,...,return_14,log_return_14,abs_log_return_14,price_28,return_28,log_return_28,abs_log_return_28,volatility_7,volatility_14,volatility_28
9263,2018-01-02,10104,68389X10,68389X10,ORCL,11.0,1,1,1,ORACLE CORP,...,4.231880,0.131071,0.131071,48.15,0.334682,0.011619,0.011619,0.136136,0.109282,0.276310
9264,2018-01-03,10104,68389X10,68389X10,ORCL,11.0,1,1,1,ORACLE CORP,...,2.890159,0.107382,0.107382,48.49,0.157137,0.006475,0.006475,0.091648,0.081082,0.268724
9265,2018-01-04,10104,68389X10,68389X10,ORCL,11.0,1,1,1,ORACLE CORP,...,2.436310,0.097613,0.097613,49.45,0.263850,0.009651,0.009651,0.089939,0.079524,0.273385
9266,2018-01-05,10104,68389X10,68389X10,ORCL,11.0,1,1,1,ORACLE CORP,...,3.586591,0.120486,0.120486,50.64,0.483153,0.015437,0.015437,0.105015,0.107098,0.281463
9267,2018-01-08,10104,68389X10,68389X10,ORCL,11.0,1,1,1,ORACLE CORP,...,1.875391,0.083731,0.083731,50.71,0.366701,0.012476,0.012476,0.105560,0.139328,0.280139
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7656951,2023-12-22,93436,88160R10,88160R10,TSLA,11.0,1,1,3,TESLA INC,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7656952,2023-12-26,93436,88160R10,88160R10,TSLA,11.0,1,1,3,TESLA INC,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7656953,2023-12-27,93436,88160R10,88160R10,TSLA,11.0,1,1,3,TESLA INC,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7656954,2023-12-28,93436,88160R10,88160R10,TSLA,11.0,1,1,3,TESLA INC,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [14]:
print(f"Number of PERMNOs: {df['permno_id'].nunique()}")
print(f"Dataset shape: {df.shape}")

Number of PERMNOs: 554
Dataset shape: (809883, 54)


In [15]:
# Get the 20 most frequent permno_ids
top_20_permnos = df['permno_id'].value_counts().nlargest(20).index.tolist()

# Filter the DataFrame to include only these 20 permno_ids
df_filtered = df[df['permno_id'].isin(top_20_permnos)]

print(f"Original dataset shape: {df.shape}")
print(f"Filtered dataset shape: {df_filtered.shape}")

Original dataset shape: (809883, 54)
Filtered dataset shape: (30180, 54)


In [16]:
# Initialize the OneHotEncoder
onehot_encoder = OneHotEncoder(sparse_output=False)

# Fit and transform the 'permno_id' column
permno_encoded = onehot_encoder.fit_transform(df_filtered[['permno_id']])

# Get the feature names for the encoded columns
encoded_feature_names = onehot_encoder.get_feature_names_out(['permno_id'])

# Create a new DataFrame with the encoded columns
df_encoded = pd.DataFrame(permno_encoded, columns=encoded_feature_names, index=df_filtered.index)

# Concatenate the original DataFrame (without 'permno_id') and the encoded DataFrame
df_final = pd.concat([df_filtered.drop('permno_id', axis=1), df_encoded], axis=1)

print(f"Number of remaining PERMNOs: {df_filtered['permno_id'].nunique()}")
print(f"Final dataset shape: {df_final.shape}")

Number of remaining PERMNOs: 20
Final dataset shape: (30180, 73)


In [18]:
df_final

,date,ncusip_id,cusip_id,ticker,share_code,share_code_type,share_code_detail,exchange_code,company_name,primary_exchange,...,permno_id_90305,permno_id_90454,permno_id_90455,permno_id_90505,permno_id_90520,permno_id_90601,permno_id_90664,permno_id_90808,permno_id_93429,permno_id_93436
9263,2018-01-02,68389X10,68389X10,ORCL,11.0,1,1,1,ORACLE CORP,N,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9264,2018-01-03,68389X10,68389X10,ORCL,11.0,1,1,1,ORACLE CORP,N,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9265,2018-01-04,68389X10,68389X10,ORCL,11.0,1,1,1,ORACLE CORP,N,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9266,2018-01-05,68389X10,68389X10,ORCL,11.0,1,1,1,ORACLE CORP,N,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9267,2018-01-08,68389X10,68389X10,ORCL,11.0,1,1,1,ORACLE CORP,N,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7656951,2023-12-22,88160R10,88160R10,TSLA,11.0,1,1,3,TESLA INC,Q,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
7656952,2023-12-26,88160R10,88160R10,TSLA,11.0,1,1,3,TESLA INC,Q,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
7656953,2023-12-27,88160R10,88160R10,TSLA,11.0,1,1,3,TESLA INC,Q,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
7656954,2023-12-28,88160R10,88160R10,TSLA,11.0,1,1,3,TESLA INC,Q,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


## Verify One Ticker

In [19]:
# Check in the original dataset
original_orcl_count = df[df['ticker'] == 'ORCL'].shape[0]
print(f"Number of ORCL instances in the original dataset: {original_orcl_count}")

# Check in the filtered dataset
filtered_orcl_count = df_filtered[df_filtered['ticker'] == 'ORCL'].shape[0]
print(f"Number of ORCL instances in the filtered dataset: {filtered_orcl_count}")

# Verify if they are the same
if original_orcl_count == filtered_orcl_count:
    print("All ORCL instances were maintained in the filtered dataset.")
else:
    print(f"Warning: Number of ORCL instances changed. {original_orcl_count - filtered_orcl_count} instances were lost.")

# You can also check the unique permno_id for ORCL
orcl_permno = df[df['ticker'] == 'ORCL']['permno_id'].unique()
print(f"ORCL permno_id: {orcl_permno}")

# Verify this permno_id is in the list of top 20 permnos
if orcl_permno[0] in top_20_permnos:
    print("ORCL's permno_id is in the top 20 list.")
else:
    print("Warning: ORCL's permno_id is not in the top 20 list.")

# You can also check the one-hot encoded column for ORCL
orcl_column = f"permno_id_{orcl_permno[0]}"
if orcl_column in df_final.columns:
    print(f"One-hot encoded column for ORCL ({orcl_column}) is present in the final dataset.")
else:
    print(f"Warning: One-hot encoded column for ORCL ({orcl_column}) is not in the final dataset.")

Number of ORCL instances in the original dataset: 1509
Number of ORCL instances in the filtered dataset: 1509
All ORCL instances were maintained in the filtered dataset.
ORCL permno_id: ['10104']
ORCL's permno_id is in the top 20 list.
One-hot encoded column for ORCL (permno_id_10104) is present in the final dataset.


In [20]:
output_file = '20_stock_data.csv'

# Save the DataFrame to a CSV file
df_final.to_csv(output_file, index=False)

print(f"Data has been saved to {output_file}")

Data has been saved to 20_stock_data.csv
